In [1]:
from fastai.text import *
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
import sentencepiece as spm
import re
import pdb

In [2]:
import fastai, torch
fastai.__version__ , torch.__version__

('1.0.61', '1.0.0')

In [3]:
torch.cuda.set_device(0)

In [4]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value)  
    torch.manual_seed(seed_value)  
    random.seed(seed_value)
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

In [5]:
random_seed(42, True)

In [6]:
path = Path('./')

In [7]:
!pwd

/data/home/ubuntu/gaurav/in/nlp-for-hindi/classification-bbc-news


In [8]:
df_train = pd.read_csv(path/'../../classification_public_datasets/iitp-product-reviews/hi/hi-train.csv', header=None)
df_train.head()

,0,1
0,negative,एंडराॅयड के मामले में यह थोड़ा पीछे है ।
1,neutral,यह एस्पेक्ट रेशो का ईश्यू है और हम आशा करते है...
2,positive,लेकिन इस तरह के एक मॉडल के एक घर कंप्यूटर के ल...
3,positive,गिर वन राष्ट्रीय उद्यान बाघ संरक्षित क्षेत्र ह...
4,neutral,"और हां , इस फिल्म में हर किरदार भारद्वाज को भर..."


In [9]:
df_valid = pd.read_csv(path/'../../classification_public_datasets/iitp-product-reviews/hi/hi-valid.csv', header=None)
df_valid.head()

,0,1
0,positive,"एक ऐसा भी जानवर है , जिसे आप पसंद तो करते होंग..."
1,neutral,इतना नहीं बल्कि यहां पर कंपनी अपना पहला फाइव स...
2,neutral,"डिजिफलीप प्रो XT811 , 1200 x 800 P और 8 इंच डि..."
3,positive,निकोन डी810 कैमरे की सबसे खास बात ये है कि इसम...
4,positive,"इसमें विंडोज 8 64-बिट ऑपरेटिंग सिस्टम है , लेक..."


In [10]:
df_test = pd.read_csv(path/'../../classification_public_datasets/iitp-product-reviews/hi/hi-test.csv', header=None)
df_test.head()

,0,1
0,neutral,"मगर , इस तरफ कोई ध्यान नहीं दिया जा रहा ।"
1,negative,गेम कई बार मुश्किल मालूम देता है ।
2,negative,मल्टीप्लेयर में छोटी स्टोरीलाइन है ।
3,positive,हाई - एंड एचपी नोटबुक्स की लुक्स हमेशा बेहतरीन...
4,positive,कैमरे बनाने वाली कंपनी निकोन अपनी 1 सीरीज कैमर...


In [11]:
df_train.shape, df_valid.shape, df_test.shape

((4182, 2), (523, 2), (523, 2))

In [12]:
df_train[df_train[1].isnull()]

,0,1


In [13]:
# df_train = df_train.dropna()

In [14]:
df_test[df_test[1].isnull()]

,0,1


In [15]:
# df_test = df_test.dropna()

In [16]:
label_cols = [0]

In [17]:
class HindiTokenizer(BaseTokenizer):
    def __init__(self, lang:str):
        self.lang = lang
        self.sp = spm.SentencePieceProcessor()
        self.sp.Load(str('./../../models/hindi/tokenizer/Created from Hindi Wikipedia Articles - 172k/hindi_lm_large.model'))
        
    def tokenizer(self, t:str) -> List[str]:
        return self.sp.EncodeAsPieces(t)

In [18]:
sp = spm.SentencePieceProcessor()
sp.Load('./../../models/hindi/tokenizer/Created from Hindi Wikipedia Articles - 172k/hindi_lm_large.model')
itos = [sp.IdToPiece(int(i)) for i in range(30000)]

In [19]:
itos[:10]

['<unk>', '<s>', '</s>', '▁के', '।', '▁में', '▁है', ',', '▁की', '▁']

In [20]:
# 30,000 is the vocab size that we chose in sentencepiece
hindi_vocab = Vocab(itos)

In [21]:
tokenizer = Tokenizer(tok_func=HindiTokenizer, lang='hi')

In [22]:
tokenizer.special_cases

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep']

In [23]:
data_lm = TextLMDataBunch.from_df(path=path, train_df=df_train, valid_df=df_valid, test_df=df_test, tokenizer=tokenizer, vocab=hindi_vocab, bs=32)

In [24]:
data_lm.show_batch()

idx,text
0,"▁ । ▁x x bo s ▁गिर ▁वन ▁राष्ट्रीय ▁उद्यान ▁बाघ ▁संरक्षित ▁क्षेत्र ▁है ▁जो ▁एशियाई ▁बब्बर ▁शेर ▁के ▁लिए ▁विश्व ▁प्रसिद्ध ▁है ▁ । ▁x x bo s ▁और ▁हां ▁ , ▁इस ▁फिल्म ▁में ▁हर ▁किरदार ▁भारद्वाज ▁को ▁भरद्वाज ▁क्यों ▁बुला ता ▁है ▁? ▁x x bo s ▁फोन ▁उपयोग ▁में ▁बेहद ▁ही ▁आसान ▁है ▁और ▁टच ▁रि स्प ॉर न्स ▁उतना ▁ही ▁बेहतर ▁ । ▁x x"
1,"▁के ▁कारण ▁ , ▁स्क्रीन ▁आँख ो ▁के ▁लिए ▁सुख कारक ▁है ▁जो ▁पढ़ने ▁और ▁वी डी यो ▁देखने ▁के ▁लिए ▁बहुत ▁अच्छी ▁है ▁ । ▁x x bo s ▁इस ▁फै ब लेट ▁में ▁बड़ा ▁बैटरी ▁भी ▁लगा ▁है ▁जो ▁बिना ▁बैटरी ▁से विंग ▁मोड ▁को ▁ एक्टिव ▁किए ▁ही ▁एक ▁दिन ▁चल ▁सकता ▁है ▁ । ▁x x bo s ▁अब ▁तक ▁ , ▁इस ▁बजट ▁में ▁खरीदने"
2,▁इनमें ▁अंतर ▁एंड्रॉइड ▁वर्जन ▁और ▁स्क्रीन ▁साइ ज ▁का ▁है ▁ । ▁x x bo s ▁गर्म ▁होने ▁की ▁समस्या ▁है ▁ । ▁x x bo s ▁जस ी डीह ▁जंक्शन ▁भारत ▁के ▁सभी ▁प्रमुख ▁भागों ▁से ▁जुड़ा ▁है ▁ । ▁x x bo s ▁मुंबई ▁के ▁माहौल ▁में ▁अंडरवर्ल्ड ▁और ▁एन काउंट र ▁अब ▁सुर्खियों ▁के ▁शब्द ▁नहीं ▁हैं ▁ । ▁x x bo s ▁मेरी ▁12 ▁महीने ▁की
3,▁देने ▁वाला ▁प्रतीत ▁होता ▁है ▁ । ▁x x bo s ▁साइन ो जन ▁ओएस ▁की ▁यह ▁खास ▁बात ▁है ▁कि ▁अप ीय रेंस ▁के ▁कस्टम ाइजेशन ▁के ▁मामले ▁में ▁आप ▁इसमें ▁कई ▁फेरबदल ▁कर ▁सकते ▁हैं ▁ । ▁x x bo s ▁कंपनी ▁का ▁कहना ▁है ▁कि ▁खाना ▁बनाने ▁के ▁लिए ▁तैयार ▁होकर ▁उसका ▁मैसेज ▁करने ▁वाले ▁इस ▁प्रेशर ▁कू कर ▁में ▁स्मार्टफोन ▁एप ▁का ▁इस्तेमाल ▁किया ▁गया ▁है
4,"प्स ▁को ▁बहुत ▁पीछे ▁छोड़ ▁छोड़ ▁देते ▁हैं ▁ । ▁x x bo s ▁रेलमार्ग ▁- ▁निकटतम ▁रेलवे ▁स्टेशन ▁बै द्य नाथ ▁धाम ▁है ▁ । ▁x x bo s ▁यह ▁खास ▁तौर ▁से ▁एक ▁छोटे ▁स्मार्टफोन ▁के ▁स्क्रीन ▁पर ▁हा र्इ ▁क्वालिटी ▁ए प्प ▁की ▁तरह ▁नहीं ▁लगता ▁है ▁x x bo s ▁यहां ▁पर ▁प्लांट ▁की ▁2 29 ▁ , ▁मै मल ▁की ▁31 ▁ , ▁बर्ड ▁की"


In [25]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3)

In [26]:
# Loading the pretrained language model on hindi wikipedia
learn.load('../../../models/hindi/lm/ULMFiT - Trained on Hindi Wikipedia Articles - 172k/model')

LanguageLearner(data=TextLMDataBunch;

Train: LabelList (4182 items)
x: LMTextList
▁x x bo s ▁एंड र ाॅय ड ▁के ▁मामले ▁में ▁यह ▁थोड़ा ▁पीछे ▁है ▁ ।,▁x x bo s ▁यह ▁ए स्पेक्ट ▁रे शो ▁का ▁ई श ्यू ▁है ▁और ▁हम ▁आशा ▁करते ▁हैं ▁कि ▁यह ▁आने ▁वाले ▁अपडेट ▁में ▁फिक्स ▁कर ▁दिया ▁जाएगा ▁ ।,▁x x bo s ▁लेकिन ▁इस ▁तरह ▁के ▁एक ▁मॉडल ▁के ▁एक ▁घर ▁कंप्यूटर ▁के ▁लिए ▁एक ▁शानदार ▁विकल्प ▁हो ▁सकता ▁है ▁ ।,▁x x bo s ▁गिर ▁वन ▁राष्ट्रीय ▁उद्यान ▁बाघ ▁संरक्षित ▁क्षेत्र ▁है ▁जो ▁एशियाई ▁बब्बर ▁शेर ▁के ▁लिए ▁विश्व ▁प्रसिद्ध ▁है ▁ ।,▁x x bo s ▁और ▁हां ▁ , ▁इस ▁फिल्म ▁में ▁हर ▁किरदार ▁भारद्वाज ▁को ▁भरद्वाज ▁क्यों ▁बुला ता ▁है ▁?
y: LMLabelList
,,,,
Path: .;

Valid: LabelList (523 items)
x: LMTextList
▁x x bo s ▁एक ▁ऐसा ▁भी ▁जानवर ▁है ▁ , ▁जिसे ▁आप ▁पसंद ▁तो ▁करते ▁होंगे ▁ , ▁लेकिन ▁उसे ▁घर ▁में ▁रखने ▁का ▁ख्याल ▁तक ▁मन ▁में ▁नहीं ▁ला ▁सकते ▁ , ▁जैसे ▁कि ▁हाथी ▁ ।,▁x x bo s ▁इतना ▁नहीं ▁बल्कि ▁यहां ▁पर ▁कंपनी ▁अपना ▁पहला ▁फाइव ▁स्टार ▁रेटिंग ▁फ्रि ज ▁भी ▁लेकर ▁आई ▁है ▁जो ▁जल्द ▁ही ▁बाजार ▁में ▁मिलने ▁लगेगा ▁ ।,▁x 

In [27]:
# Fine tuning the prtrained LM on current dataset

In [28]:
learn.lr_find()

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


In [29]:
learn.recorder.plot()

In [30]:
learn.freeze()

In [31]:
learn.fit_one_cycle(1, 1e-2, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,3.887326,3.609842,0.422258,00:03


In [32]:
learn.save('fit_head', with_opt=True)

In [33]:
learn.load('fit_head', with_opt=True);

In [34]:
learn.unfreeze()

In [35]:
learn.fit_one_cycle(2, 1e-3, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,3.161343,3.373509,0.442793,00:03
1,2.884025,3.340018,0.448151,00:03


In [36]:
learn.save('fine_tuned', with_opt=True)

In [37]:
learn.load('fine_tuned', with_opt=True);

In [38]:
learn.predict('▁इस ▁लाइन ▁के ▁चलने ▁से ▁दक्षिणी ',n_words=10)

'▁इस ▁लाइन ▁के ▁चलने ▁से ▁दक्षिणी  ▁भारत ▁में ▁कई ▁ग्रामीण ▁आबादी ▁से ▁पैदा ▁हुए ▁लोगों ▁की'

In [39]:
learn.save_encoder('fine_tuned_enc')

In [40]:
data_clas = TextClasDataBunch.from_df(path=path, train_df=df_train, valid_df=df_valid, test_df=df_test, tokenizer=tokenizer, vocab=hindi_vocab)

In [41]:
data_clas.show_batch()

text,target
"▁x x bo s ▁नि कॉन ▁कूल ▁पि क् ▁स <unk> ▁s 6 300 <unk> ▁n ik on ▁co ol pi x <unk> ▁s 6 300 ▁कीमत ▁9 , 65 2 ▁रुपए ▁ , ▁कैमरा ▁साइ ज ▁9 3.6 ▁x ▁5 7.7 ▁x ▁36 ▁एम एम ▁ , ▁वे ट ▁: ▁160 ▁ग्राम ▁रेज ् ▁यू लू शन ▁: ▁16 ▁मेगा पि क् ▁सल ▁फो कल ▁ले ंथ ▁: ▁25",positive
"▁x x bo s ▁एप् ▁पल ▁के ▁प्रो डे क् ▁ट ▁पूरे ▁विश् ▁व ▁में ▁मशहूर ▁हैं ▁फिर ▁चाहें ▁वो ▁स् ▁ मार्ट फोन ▁हो ▁या ▁फिर ▁कोई ▁अन् ▁य ▁एस ेसी रीज ▁ , ▁एप् ▁पल ▁भ विष् ▁य ▁में ▁कुछ ▁अपने ▁गै जेट ों ▁की ▁डिजाइन ▁में ▁काफी ▁बल ाव ▁करने ▁पर ▁विचार ▁कर ▁रहा ▁है ▁इसका ▁जीता ▁जाता ▁उदाहरण ▁है ▁एप् ▁पल ▁आई वॉ च ▁2 ▁इसे",positive
▁x x bo s ▁इस ▁बारे ▁में ▁सबसे ▁इर ् म् ▁पो टेंट ▁बात ▁यह ▁है ▁कि ▁अगर ▁आप ▁सैमसंग ▁के ▁अन् ▁य ▁प्रॉ ड क् ▁ ट्स ▁को ▁भी ▁यू ज ▁कर ▁रहे ▁हैं ▁तो ▁यह ▁डिवाइस ▁आपके ▁स् ▁ मार्ट फोन ▁के ▁कॉम् ▁पेट िबल ▁रहेगी ▁लेकिन ▁अगर ▁आप ▁सैमसंग ▁यूजर ▁नही ▁हैं ▁तो ▁आप ▁थोड़ा ▁वे ट ▁कर ▁सकते ▁हैं ▁क् ▁ योंकि ▁मोटोरोला ▁भी ▁अपनी ▁स्,neutral
▁x x bo s ▁ये रक ाड ▁या ▁ यर कौ द ▁में ▁ यूँ ▁तो ▁होटलों ▁की ▁कमी ▁नहीं ▁है ▁पर ▁बीस वें ▁हेयर पिन ▁बे ंड ▁के ▁ठीक ▁बाद <unk> ▁g <unk> rt <unk> ▁n ature <unk> il s ▁में ▁अगर ▁ना ▁भी ▁रुक ें ▁तो ▁यहाँ ▁एक ▁बार ▁भोजन ▁करना ▁तो ▁बनता ▁है ▁सिर्फ ▁इसलिए ▁नहीं ▁कि ▁यहाँ ▁के ▁व्यंजन ▁ल जी ज़ ▁हैं ▁बल्कि ▁खास तौर,positive
"▁x x bo s ▁माइक्रो मैक्स ▁फ न बुक ▁की ▁एक ▁और ▁विशेषता ▁है ▁ , ▁इसमें ▁लगा ▁डू अल ▁कोर ▁400 <unk> ▁का ▁ग्राफिक ▁प्रोसेसर ▁जिससे ▁इस ▁डिवाइस ▁पर ▁फुल <unk> <unk> ▁10 80 p ▁का ▁वीडियो ▁प्लेबैक ▁की ▁सुविधा ▁भी ▁है ▁ , ▁पर ▁एक ▁बार ▁फिर ▁कम ▁डिस प्ले ▁रिज ॉ ल्यू शन ▁हाई ▁डे फिन िशन ▁कॉन् टेंट ▁के ▁म जे ▁को ▁खराब ▁कर ▁देता ▁है",positive


In [42]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)

In [43]:
learn.load_encoder('fine_tuned_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (4182 items)
x: TextList
▁x x bo s ▁एंड र ाॅय ड ▁के ▁मामले ▁में ▁यह ▁थोड़ा ▁पीछे ▁है ▁ ।,▁x x bo s ▁यह ▁ए स्पेक्ट ▁रे शो ▁का ▁ई श ्यू ▁है ▁और ▁हम ▁आशा ▁करते ▁हैं ▁कि ▁यह ▁आने ▁वाले ▁अपडेट ▁में ▁फिक्स ▁कर ▁दिया ▁जाएगा ▁ ।,▁x x bo s ▁लेकिन ▁इस ▁तरह ▁के ▁एक ▁मॉडल ▁के ▁एक ▁घर ▁कंप्यूटर ▁के ▁लिए ▁एक ▁शानदार ▁विकल्प ▁हो ▁सकता ▁है ▁ ।,▁x x bo s ▁गिर ▁वन ▁राष्ट्रीय ▁उद्यान ▁बाघ ▁संरक्षित ▁क्षेत्र ▁है ▁जो ▁एशियाई ▁बब्बर ▁शेर ▁के ▁लिए ▁विश्व ▁प्रसिद्ध ▁है ▁ ।,▁x x bo s ▁और ▁हां ▁ , ▁इस ▁फिल्म ▁में ▁हर ▁किरदार ▁भारद्वाज ▁को ▁भरद्वाज ▁क्यों ▁बुला ता ▁है ▁?
y: CategoryList
negative,neutral,positive,positive,neutral
Path: .;

Valid: LabelList (523 items)
x: TextList
▁x x bo s ▁एक ▁ऐसा ▁भी ▁जानवर ▁है ▁ , ▁जिसे ▁आप ▁पसंद ▁तो ▁करते ▁होंगे ▁ , ▁लेकिन ▁उसे ▁घर ▁में ▁रखने ▁का ▁ख्याल ▁तक ▁मन ▁में ▁नहीं ▁ला ▁सकते ▁ , ▁जैसे ▁कि ▁हाथी ▁ ।,▁x x bo s ▁इतना ▁नहीं ▁बल्कि ▁यहां ▁पर ▁कंपनी ▁अपना ▁पहला ▁फाइव ▁स्टार ▁रेटिंग ▁फ्रि ज ▁भी ▁लेकर ▁आई ▁है ▁जो ▁जल्द ▁ही ▁

In [44]:
learn.freeze()

In [45]:
learn.lr_find()

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


In [46]:
learn.recorder.plot()

In [47]:
learn.loss_func.func

CrossEntropyLoss()

In [48]:
mcc = MatthewsCorreff()

In [49]:
learn.metrics = [mcc, accuracy]

In [50]:
learn.fit_one_cycle(1, 1e-1, moms=(0.8,0.7))

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,0.855938,0.774860,0.389029,0.636711,00:01


In [51]:
learn.save('first-full')

In [52]:
learn.load('first-full');

In [53]:
learn.freeze_to(-2)
learn.fit_one_cycle(4, slice(1e-2/(2.6**4),1e-2), moms=(0.8,0.7))

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,0.755116,0.692089,0.453856,0.673040,00:01
1,0.694784,0.651466,0.479238,0.688336,00:01
2,0.588531,0.619435,0.551255,0.724665,00:01
3,0.472915,0.634819,0.553503,0.722753,00:01


In [54]:
learn.save('second-full')

In [55]:
learn.load('second-full')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (4182 items)
x: TextList
▁x x bo s ▁एंड र ाॅय ड ▁के ▁मामले ▁में ▁यह ▁थोड़ा ▁पीछे ▁है ▁ ।,▁x x bo s ▁यह ▁ए स्पेक्ट ▁रे शो ▁का ▁ई श ्यू ▁है ▁और ▁हम ▁आशा ▁करते ▁हैं ▁कि ▁यह ▁आने ▁वाले ▁अपडेट ▁में ▁फिक्स ▁कर ▁दिया ▁जाएगा ▁ ।,▁x x bo s ▁लेकिन ▁इस ▁तरह ▁के ▁एक ▁मॉडल ▁के ▁एक ▁घर ▁कंप्यूटर ▁के ▁लिए ▁एक ▁शानदार ▁विकल्प ▁हो ▁सकता ▁है ▁ ।,▁x x bo s ▁गिर ▁वन ▁राष्ट्रीय ▁उद्यान ▁बाघ ▁संरक्षित ▁क्षेत्र ▁है ▁जो ▁एशियाई ▁बब्बर ▁शेर ▁के ▁लिए ▁विश्व ▁प्रसिद्ध ▁है ▁ ।,▁x x bo s ▁और ▁हां ▁ , ▁इस ▁फिल्म ▁में ▁हर ▁किरदार ▁भारद्वाज ▁को ▁भरद्वाज ▁क्यों ▁बुला ता ▁है ▁?
y: CategoryList
negative,neutral,positive,positive,neutral
Path: .;

Valid: LabelList (523 items)
x: TextList
▁x x bo s ▁एक ▁ऐसा ▁भी ▁जानवर ▁है ▁ , ▁जिसे ▁आप ▁पसंद ▁तो ▁करते ▁होंगे ▁ , ▁लेकिन ▁उसे ▁घर ▁में ▁रखने ▁का ▁ख्याल ▁तक ▁मन ▁में ▁नहीं ▁ला ▁सकते ▁ , ▁जैसे ▁कि ▁हाथी ▁ ।,▁x x bo s ▁इतना ▁नहीं ▁बल्कि ▁यहां ▁पर ▁कंपनी ▁अपना ▁पहला ▁फाइव ▁स्टार ▁रेटिंग ▁फ्रि ज ▁भी ▁लेकर ▁आई ▁है ▁जो ▁जल्द ▁ही ▁

In [56]:
learn.unfreeze()
learn.fit_one_cycle(3, slice(1e-2/(2.6**4),1e-2), moms=(0.8,0.7))

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,0.477516,0.652740,0.532421,0.711281,00:03
1,0.405279,0.717211,0.563376,0.718929,00:03
2,0.232454,0.703717,0.568024,0.734226,00:03


In [57]:
learn.save('final')

In [59]:
from sklearn.metrics import accuracy_score, matthews_corrcoef
df_dict = {'query': list(df_test[1]), 'actual_label': list(df_test[0]), 'predicted_label': ['']*df_test.shape[0]}
all_nodes = list(set(df_train[0]))
for node in all_nodes:
    df_dict[node] = ['']*df_test.shape[0]
    
i2c = {}
for key, value in learn.data.c2i.items():
    i2c[value] = key
    
df_result = pd.DataFrame(df_dict)
for index, row in df_result.iterrows():
    pred = learn.predict(data_clas.test_ds[index])
    for node in all_nodes:
        row[node] = pred[2][learn.data.c2i[node]].item()
    row['predicted_label'] = i2c[pred[0].data.item()]
df_result.head()

,query,actual_label,predicted_label,negative,neutral,positive
0,"मगर , इस तरफ कोई ध्यान नहीं दिया जा रहा ।",neutral,neutral,0.31099,0.655697,0.0333129
1,गेम कई बार मुश्किल मालूम देता है ।,negative,negative,0.820817,0.118118,0.061065
2,मल्टीप्लेयर में छोटी स्टोरीलाइन है ।,negative,neutral,0.0787148,0.657412,0.263873
3,हाई - एंड एचपी नोटबुक्स की लुक्स हमेशा बेहतरीन...,positive,positive,6.82234e-05,0.000406989,0.999525
4,कैमरे बनाने वाली कंपनी निकोन अपनी 1 सीरीज कैमर...,positive,positive,0.000679931,0.0035804,0.99574


In [60]:
accuracy_score(df_result['actual_label'], df_result['predicted_label'])

0.7571701720841301

In [61]:
matthews_corrcoef(df_result['actual_label'], df_result['predicted_label'])

0.5976115429660378

In [62]:
df_result.to_csv('iitp_product_result.csv', index=False)